In [1]:
from huggingface_hub import login
from google.colab import userdata
HF_TOKEN=userdata.get('huggingface')

In [2]:
!pip install datasets==3.6.0
!pip install evaluate
!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=8b089d9f40a6becdc40516be18def8335a86b34a3e10b09700ac32b2f2b8cd9a
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM

data = load_dataset("AmazonScience/massive", "de-DE")
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-german-cased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-german-cased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

massive.py: 0.00B [00:00, ?B/s]

0000.parquet:   0%|          | 0.00/815k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/163k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-german-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Предобработка


In [4]:
idx_to_intent = data["train"].features["intent"].names
print(f"Количество интентов: {len(idx_to_intent)}")
print(f"Примеры интентов: {idx_to_intent[:5]}")

Количество интентов: 60
Примеры интентов: ['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock']


In [5]:
print("Пример из датасета:")
example = data["train"][0]
print(f"utt: {example['utt']}")
print(f"intent: {example['intent']} - {idx_to_intent[example['intent']]}")
print(f"annot_utt: {example['annot_utt']}")

Пример из датасета:
utt: weck mich am freitag um neun uhr auf
intent: 48 - alarm_set
annot_utt: weck mich am [date : freitag] um [time : neun uhr] auf


In [6]:
import re

def annot_to_bio(annot_utt_string):
    bio_tags = []
    tokens_list = re.findall(r'\[[^\]]+\]|\S+', annot_utt_string)

    for token_item in tokens_list:
        if token_item.startswith('[') and token_item.endswith(']'):
            slot_parts = token_item[1:-1].split(':', 1)
            slot_name = slot_parts[0].strip()
            slot_value_str = slot_parts[1].strip() if len(slot_parts) > 1 else ""
            # Разделяем значение на слова, чтобы разметить каждое слово
            value_words = slot_value_str.split()

            bio_tags.append(f"B-{slot_name}")
            for _ in range(1, len(value_words)):
                bio_tags.append(f"I-{slot_name}")
        else:
            bio_tags.append("O")

    return bio_tags

In [7]:
example = data["train"][0]
print(f"Оригинальная аннотация: {example['annot_utt']}")
bio_tags = annot_to_bio(example['annot_utt'])
print(f"BIO теги: {bio_tags}")

Оригинальная аннотация: weck mich am [date : freitag] um [time : neun uhr] auf
BIO теги: ['O', 'O', 'O', 'B-date', 'O', 'B-time', 'I-time', 'O']


In [8]:
# Собираем все уникальные BIO теги из датасета
all_bio_tags = set()

for split in ["train", "validation", "test"]:
    for example in data[split]:
        bio_tags = annot_to_bio(example['annot_utt'])
        all_bio_tags.update(bio_tags)

# Добавляем все возможные I-теги для каждого B-тега на случай,
# если tokenizer разбил слово на несколько подтокенов
newly_added_i_tags = set()
for tag in list(all_bio_tags):
    if tag.startswith("B-"):
        i_tag = "I-" + tag[2:]
        if i_tag not in all_bio_tags:
            newly_added_i_tags.add(i_tag)

all_bio_tags.update(newly_added_i_tags)

# Сортируем теги: O -> B- -> I-
bio_tags_list = sorted(list(all_bio_tags), key=lambda x: (x != "O", x))

print(f"Количество уникальных BIO тегов: {len(bio_tags_list)}")
print(f"Примеры тегов: {bio_tags_list[:10]}")

# Словари для маппинга
tag2id = {tag: idx for idx, tag in enumerate(bio_tags_list)}
id2tag = {idx: tag for tag, idx in tag2id.items()}

Количество уникальных BIO тегов: 111
Примеры тегов: ['O', 'B-alarm_type', 'B-app_name', 'B-artist_name', 'B-audiobook_author', 'B-audiobook_name', 'B-business_name', 'B-business_type', 'B-change_amount', 'B-coffee_type']


In [14]:
def tokenize_and_align_labels(examples, tokenizer, tag2id):
    # Разбиваем utt строку на слова
    words_per_utterance = [utt.split() for utt in examples["utt"]]

    tokenized_inputs = tokenizer(
        words_per_utterance,
        truncation=True,
        padding=False,
        is_split_into_words=True
    )

    labels_intent = []
    labels_ner = []

    for i, example_annot in enumerate(examples["annot_utt"]):
        current_utt_words = words_per_utterance[i]
        bio_tags_from_annot = annot_to_bio(example_annot)

        # Проверяем, что количество тегов совпадает с количеством слов
        if len(bio_tags_from_annot) != len(current_utt_words):
            print(f"Несоответствие количества тегов и слов (индекс {i}).")
            print(f"Оригинальный utt: {examples['utt'][i]}")
            print(f"Слова (из разбиения): {current_utt_words} (длина: {len(current_utt_words)})\n")
            print(f"Аннотированный utt: {example_annot}")
            print(f"BIO тэги: {bio_tags_from_annot} (длина: {len(bio_tags_from_annot)})\n")

            if len(bio_tags_from_annot) < len(current_utt_words):
                bio_tags = bio_tags_from_annot + ['O'] * (len(current_utt_words) - len(bio_tags_from_annot))
            elif len(bio_tags_from_annot) > len(current_utt_words):
                bio_tags = bio_tags_from_annot[:len(current_utt_words)]
        else:
            bio_tags = bio_tags_from_annot

        # Маппим токены на слова
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        # Выравниваем метки
        label_ids = []
        previous_word_idx = None

        for word_idx in word_ids:
            # Спец. токены = -100
            if word_idx is None:
                label_ids.append(-100)
            # Первый токен слова получает метку B-
            elif word_idx != previous_word_idx:
                if word_idx < len(bio_tags):
                    label_ids.append(tag2id[bio_tags[word_idx]])
                else:
                    print(f"Ошибка: word_idx ({word_idx}) вышел за границы bio_tags (длина {len(bio_tags)}) для примера {i}.")
                    label_ids.append(-100)
            else:
                # А последующие токены того же слова получают I-
                if word_idx < len(bio_tags):
                    tag = bio_tags[word_idx]
                    if tag.startswith("B-"):
                        tag = "I-" + tag[2:]
                    label_ids.append(tag2id[tag])
                else:
                    label_ids.append(-100)

            previous_word_idx = word_idx

        labels_ner.append(label_ids)
        # Для классификации интентов просто берем индекс интента
        labels_intent.append(examples["intent"][i])

    tokenized_inputs["labels_ner"] = labels_ner
    tokenized_inputs["labels_intent"] = labels_intent

    return tokenized_inputs

In [17]:
test_example = {
    "utt": [data["train"][0]["utt"]],
    "annot_utt": [data["train"][0]["annot_utt"]],
    "intent": [data["train"][0]["intent"]],
}

tokenized = tokenize_and_align_labels(test_example, tokenizer, tag2id)

print("Токены:", tokenizer.convert_ids_to_tokens(tokenized["input_ids"][0]))
print(f"\nNER метки: {tokenized['labels_ner'][0]}")
print(f"Intent метка: {tokenized['labels_intent'][0]} -> {idx_to_intent[tokenized['labels_intent'][0]]}")

print("\nВыравнивание токенов и меток:")
tokens = tokenizer.convert_ids_to_tokens(tokenized["input_ids"][0])
ner_labels = tokenized['labels_ner'][0]
for token, label_id in zip(tokens, ner_labels):
    if label_id != -100:
        print(f"{token:20} -> {id2tag[label_id]}")
    else:
        print(f"{token:20} -> [SPECIAL]")

Токены: ['[CLS]', 'we', '##ck', 'mich', 'am', 'frei', '##tag', 'um', 'neun', 'u', '##hr', 'auf', '[SEP]']

NER метки: [-100, 0, 0, 0, 0, 13, 68, 0, 48, 103, 103, 0, -100]
Intent метка: 48 -> alarm_set

Выравнивание токенов и меток:
[CLS]                -> [SPECIAL]
we                   -> O
##ck                 -> O
mich                 -> O
am                   -> O
frei                 -> B-date
##tag                -> I-date
um                   -> O
neun                 -> B-time
u                    -> I-time
##hr                 -> I-time
auf                  -> O
[SEP]                -> [SPECIAL]


In [19]:
# Применяем препроцессинг ко всему датасету
from functools import partial

tokenize_function = partial(
    tokenize_and_align_labels,
    tokenizer=tokenizer,
    tag2id=tag2id,
)

# Применяем к каждому разделу датасета
tokenized_datasets = data.map(
    tokenize_function,
    batched=True,
    remove_columns=data["train"].column_names,
    desc="Токенизация и выравнивание меток"
)

print("Токенизированные датасеты:")
print(tokenized_datasets)
print(f"\nПример токенизированного образца:")
print(f"Input IDs shape: {len(tokenized_datasets['train'][0]['input_ids'])}")
print(f"NER labels shape: {len(tokenized_datasets['train'][0]['labels_ner'])}")
print(f"Intent label: {tokenized_datasets['train'][0]['labels_intent']}")

Токенизация и выравнивание меток:   0%|          | 0/11514 [00:00<?, ? examples/s]

Токенизация и выравнивание меток:   0%|          | 0/2033 [00:00<?, ? examples/s]

Токенизация и выравнивание меток:   0%|          | 0/2974 [00:00<?, ? examples/s]

Токенизированные датасеты:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels_ner', 'labels_intent'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels_ner', 'labels_intent'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels_ner', 'labels_intent'],
        num_rows: 2974
    })
})

Пример токенизированного образца:
Input IDs shape: 13
NER labels shape: 13
Intent label: 48


Дообучение модели на NER и классификацию интентов

In [20]:
import torch
import torch.nn as nn
from transformers import (
    BertPreTrainedModel,
    BertModel,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
import numpy as np
from sklearn.metrics import f1_score
import evaluate

seqeval = evaluate.load("seqeval")

In [49]:
class BertMultiTaskModel(BertPreTrainedModel):
    def __init__(self, config, num_intent_labels, num_ner_labels):
        super().__init__(config)
        self.num_intent_labels = num_intent_labels
        self.num_ner_labels = num_ner_labels

        # Общий BERT энкодер
        self.bert = BertModel(config)
        # Регуляризация
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # Голова для классификации интентов
        self.intent_classifier = nn.Linear(config.hidden_size, num_intent_labels)
        # Голова для NER
        self.ner_classifier = nn.Linear(config.hidden_size, num_ner_labels)
        # Инициализация весов
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels_intent=None,
        labels_ner=None,
        **kwargs,
    ):
        # Прогон через BERT
        outputs = self.bert(input_ids, attention_mask, token_type_ids)

        sequence_output = outputs[0] # скрытые предст-я всех токенов
        pooled_output = outputs[1] # вектор [CLS], представляющий весь вход

        # Применяем dropout
        sequence_output = self.dropout(sequence_output)
        pooled_output = self.dropout(pooled_output)

        # Интенты - 1 логит-вектор на каждый пример
        intent_logits = self.intent_classifier(pooled_output)
        # NER - логит на каждый токен
        ner_logits = self.ner_classifier(sequence_output)

        total_loss = None
        intent_loss = None
        ner_loss = None

        if labels_intent is not None and labels_ner is not None:
            loss_fct_intent = nn.CrossEntropyLoss()
            intent_loss = loss_fct_intent(
                intent_logits.view(-1, self.num_intent_labels),
                labels_intent.view(-1)
            )

            loss_fct_ner = nn.CrossEntropyLoss(ignore_index=-100)
            ner_loss = loss_fct_ner(
                ner_logits.view(-1, self.num_ner_labels),
                labels_ner.view(-1)
            )
            total_loss = intent_loss + ner_loss

        return {
            'loss': total_loss,
            'intent_loss': intent_loss,
            'ner_loss': ner_loss,
            'intent_logits': intent_logits,
            'ner_logits': ner_logits,
            'hidden_states': outputs.hidden_states if hasattr(outputs, 'hidden_states') else None,
            'attentions': outputs.attentions if hasattr(outputs, 'attentions') else None,
        }

In [57]:
# Создаем инстанс модели
config = AutoConfig.from_pretrained("google-bert/bert-base-german-cased")

num_intent_labels = len(idx_to_intent)
num_ner_labels = len(tag2id)

print(f"Количество классов интентов: {num_intent_labels}")
print(f"Количество NER меток: {num_ner_labels}")

model = BertMultiTaskModel.from_pretrained(
    "google-bert/bert-base-german-cased",
    config=config,
    num_intent_labels=num_intent_labels,
    num_ner_labels=num_ner_labels,
    ignore_mismatched_sizes=True,  # разрешаем загрузку весов для наших голов
)

Some weights of BertMultiTaskModel were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['intent_classifier.bias', 'intent_classifier.weight', 'ner_classifier.bias', 'ner_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Количество классов интентов: 60
Количество NER меток: 111


In [44]:
from dataclasses import dataclass
from typing import Dict, List
from transformers import PreTrainedTokenizerBase

@dataclass
class MultiTaskDataCollator:
    """
    Data collator для многозадачного обучения.
    Обрабатывает паддинг и создает батчи для обеих задач.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: bool = True
    max_length: int = None
    pad_to_multiple_of: int = None

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        # features - список словарей, каждый словарь - один обучающий пример
        batch = {}
        input_ids = [f["input_ids"] for f in features]
        attention_mask = [f["attention_mask"] for f in features]
        max_len = max(len(ids) for ids in input_ids)

        padded_input_ids = []
        padded_attention_mask = []
        padded_ner_labels = []

        # Каждый пример в батче выравниваем паддингами до длины самого длинного примера в батче
        for i, feature in enumerate(features):
            padding_length = max_len - len(feature["input_ids"])
            padded_input_ids.append(feature["input_ids"] + [self.tokenizer.pad_token_id] * padding_length)
            padded_attention_mask.append(feature["attention_mask"] + [0] * padding_length)
            padded_ner_labels.append(feature["labels_ner"] + [-100] * padding_length)

        batch["input_ids"] = torch.tensor(padded_input_ids, dtype=torch.long)
        batch["attention_mask"] = torch.tensor(padded_attention_mask, dtype=torch.long)
        batch["labels_ner"] = torch.tensor(padded_ner_labels, dtype=torch.long)
        batch["labels_intent"] = torch.tensor([f["labels_intent"] for f in features], dtype=torch.long)

        return batch

data_collator = MultiTaskDataCollator(tokenizer)

In [45]:
def compute_metrics(eval_pred):
    """
    Вычисляет метрики для обеих задач: intent classification и NER
    - Для интентов: accuracy и F1
    - Для NER: accuracy и F1 из seqeval
    - End-to-end accuracy: доля примеров, где полностью совпали интенты и все сущности
    """
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids

    # predictions - кортеж (intent_logits, ner_logits)
    intent_logits, ner_logits = predictions
    # labels - кортеж (intent_labels, ner_labels)
    intent_labels, ner_labels = labels

    results = {}

    # Метрики для интентов
    intent_predictions = np.argmax(intent_logits, axis=-1)
    intent_accuracy = np.mean(intent_predictions == intent_labels)
    results["intent_accuracy"] = intent_accuracy
    intent_f1 = f1_score(intent_labels, intent_predictions, average='macro', zero_division=0)
    results["intent_f1"] = intent_f1

    # Метрики для NER
    ner_predictions = np.argmax(ner_logits, axis=-1)
    # Для каждого примера в батче проходим по всем токенам в примере, кроме паддингов
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(ner_predictions, ner_labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(ner_predictions, ner_labels)
    ]
    ner_results = seqeval.compute(predictions=true_predictions, references=true_labels)
    results["ner_accuracy"] = ner_results["overall_accuracy"]
    results["ner_f1"] = ner_results["overall_f1"]

    # End-to-end accuracy
    end_to_end_correct = 0
    for i in range(len(intent_predictions)):
        # Проверяем совпадение интента
        intent_match = intent_predictions[i] == intent_labels[i]
        # Проверяем полное совпадение последовательностей NER
        ner_match = true_predictions[i] == true_labels[i]
        if intent_match and ner_match:
            end_to_end_correct += 1

    end_to_end_accuracy = end_to_end_correct / len(intent_predictions)
    results["end_to_end_accuracy"] = end_to_end_accuracy

    return results

In [51]:
class MultiTaskTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels_intent = inputs.pop("labels_intent")
        labels_ner = inputs.pop("labels_ner")
        outputs = model(**inputs, labels_intent=labels_intent, labels_ner=labels_ner)
        loss = outputs["loss"]

        if self.state.global_step % self.args.logging_steps == 0:
            if outputs["intent_loss"] is not None:
                self.log({"intent_loss": outputs["intent_loss"].item()})
            if outputs["ner_loss"] is not None:
                self.log({"ner_loss": outputs["ner_loss"].item()})

        return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        labels_intent = inputs.pop("labels_intent")
        labels_ner = inputs.pop("labels_ner")

        model.eval()
        with torch.no_grad():
            outputs = model(**inputs, labels_intent=labels_intent, labels_ner=labels_ner)
            loss = outputs["loss"]
            intent_logits = outputs["intent_logits"]
            ner_logits = outputs["ner_logits"]

        return (
            loss,
            (intent_logits, ner_logits),
            (labels_intent, labels_ner),
        )

In [58]:
# Параметры обучения
training_args = TrainingArguments(
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs_multitask",
    push_to_hub=False, # не пушить на hugging face
    remove_unused_columns=False,  # не даем трейнеру удалять колонки (может сломать многозадачность)
)

trainer = MultiTaskTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2779969601.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultiTaskTrainer.__init__`. Use `processing_class` instead.
  trainer = MultiTaskTrainer(


Обучение

In [59]:
print("\nОбучение началось...")
trainer.train()
print("\n Обучение завершено!")


Обучение началось...


Epoch,Training Loss,Validation Loss,Intent Accuracy,Intent F1,Ner Accuracy,Ner F1,End To End Accuracy
1,No log,2.008277,0.693064,0.485122,0.848599,0.488706,0.356616
2,3.139700,1.357093,0.795376,0.662017,0.884246,0.612431,0.503197
3,1.321900,1.156117,0.818003,0.716055,0.896029,0.655139,0.553369
4,1.321900,1.115491,0.830792,0.764288,0.903951,0.693878,0.586818
5,0.818300,1.079048,0.840138,0.778907,0.907763,0.696941,0.600590
6,0.583300,1.098275,0.844073,0.812856,0.908605,0.718395,0.613871
7,0.426400,1.107436,0.846040,0.803812,0.909199,0.712057,0.610428
8,0.426400,1.109424,0.840630,0.799551,0.912021,0.727490,0.622725
9,0.332200,1.124271,0.841613,0.810377,0.911476,0.724596,0.619282
10,0.286200,1.128892,0.842597,0.813190,0.910486,0.727057,0.620758


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco


 Обучение завершено!


In [60]:
# Оценка модели на тестовом наборе
test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])

print("\n" + "="*60)
print("Метрики на тесте")
print("="*60)
print(f"\nКлассификация интентов:")
print(f"  Accuracy: {test_results.get('eval_intent_accuracy', 0):.4f}")
print(f"  F1-score: {test_results.get('eval_intent_f1', 0):.4f}")
print(f"\nNamed Entity Recognition:")
print(f"  Accuracy: {test_results.get('eval_ner_accuracy', 0):.4f}")
print(f"  F1-score: {test_results.get('eval_ner_f1', 0):.4f}")
print(f"\nEnd-to-end оценка:")
print(f"  Accuracy: {test_results.get('eval_end_to_end_accuracy', 0):.4f}")


Метрики на тесте

Классификация интентов:
  Accuracy: 0.8500
  F1-score: 0.7894

Named Entity Recognition:
  Accuracy: 0.9059
  F1-score: 0.7084

End-to-end оценка:
  Accuracy: 0.6184


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
def predict_multitask(text, tokenizer, model, id2tag, idx_to_intent):
    # Токенизация
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Перемещаем входные тензоры на то же устройство, что и модель (GPU, если доступно)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Предсказание
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

        # Получаем предсказания для интентов
        intent_logits = outputs["intent_logits"]
        predicted_intent_id = torch.argmax(intent_logits, dim=1).item()
        predicted_intent = idx_to_intent[predicted_intent_id]
        # Получаем предсказания для NER
        ner_logits = outputs["ner_logits"]
        predicted_ner_ids = torch.argmax(ner_logits, dim=2)[0].tolist()

    # Получаем токены
    # перемещаем input_ids обратно на CPU для convert_ids_to_tokens, если они были на GPU
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].cpu()[0])

    # Фильтруем специальные токены и получаем NER метки
    ner_tags = []
    filtered_tokens = []
    for token, ner_id in zip(tokens, predicted_ner_ids):
        if token not in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            filtered_tokens.append(token)
            ner_tags.append(id2tag[ner_id])

    return predicted_intent, filtered_tokens, ner_tags


def process_user_query(text, tokenizer, model, id2tag, idx_to_intent):
    # Получаем предсказания от модели
    predicted_intent, tokens, ner_tags = predict_multitask(
        text, tokenizer, model, id2tag, idx_to_intent
    )

    # Преобразуем BIO-теги в слоты
    slots = []
    current_slot_name = None
    current_slot_tokens = []

    for token, tag in zip(tokens, ner_tags):
        if tag == "O":
            # Если встретили O, завершаем текущий слот (если есть)
            if current_slot_name is not None:
                # Объединяем токены в значение слота
                slot_value = tokenizer.convert_tokens_to_string(current_slot_tokens)
                slots.append(f"{current_slot_name}: {slot_value}")
                current_slot_name = None
                current_slot_tokens = []
        elif tag.startswith("B-"):
            # Начало нового слота
            # Если был предыдущий слот, завершаем его
            if current_slot_name is not None:
                slot_value = tokenizer.convert_tokens_to_string(current_slot_tokens)
                slots.append(f"{current_slot_name}: {slot_value}")

            # Начинаем новый слот
            current_slot_name = tag[2:]  # Убираем префикс "B-"
            current_slot_tokens = [token]
        elif tag.startswith("I-"):
            # Продолжение текущего слота
            slot_name = tag[2:]  # Убираем префикс "I-"
            if current_slot_name == slot_name:
                # Продолжаем тот же слот
                current_slot_tokens.append(token)
            else:
                # Несоответствие: I-тег без B-тега, начинаем новый слот
                if current_slot_name is not None:
                    slot_value = tokenizer.convert_tokens_to_string(current_slot_tokens)
                    slots.append(f"{current_slot_name}: {slot_value}")
                current_slot_name = slot_name
                current_slot_tokens = [token]

    # Завершаем последний слот, если он есть
    if current_slot_name is not None:
        slot_value = tokenizer.convert_tokens_to_string(current_slot_tokens)
        slots.append(f"{current_slot_name}: {slot_value}")

    return predicted_intent, slots

Тест на пользовательском тексте

In [62]:
print("\n" + "=" * 80)
print("Тест на пользовательском запросе")
print("=" * 80)

test_texts = [
    "Ich möchte einen Tisch im Restaurant buchen",
    "Wie wird das Wetter morgen in Berlin",
    "Spiele Musik von Mozart ab",
]

for text in test_texts:
    intent_name, slots = process_user_query(text, tokenizer, model, id2tag, idx_to_intent)
    print(f"\n{'='*80}")
    print(f"Текст: {text}")
    print(f"\nНамерение: {intent_name}")
    print(f"\nСлоты:")
    if slots:
        for slot in slots:
            print(f"   {slot}")
    else:
        print(f"   (нет слотов)")


Тест на пользовательском запросе

Текст: Ich möchte einen Tisch im Restaurant buchen

Намерение: transport_ticket

Слоты:
   (нет слотов)

Текст: Wie wird das Wetter morgen in Berlin

Намерение: weather_query

Слоты:
   date: morgen
   place_name: Berlin

Текст: Spiele Musik von Mozart ab

Намерение: play_music

Слоты:
   (нет слотов)
